## Timeseries

Wir arbeiten mit Zeitserien, Diskretisierung in Intervalle, zB 15 Minuten Werte.

Python Tool: pandas DataFrame mit DateTimeIndex. 

In [3]:
import pandas as pd

dti = pd.date_range('2020-01-01 00:00:00', '2021-01-01 00:00:00', tz='utc', freq='15min')
dti

DatetimeIndex(['2020-01-01 00:00:00+00:00', '2020-01-01 00:15:00+00:00',
               '2020-01-01 00:30:00+00:00', '2020-01-01 00:45:00+00:00',
               '2020-01-01 01:00:00+00:00', '2020-01-01 01:15:00+00:00',
               '2020-01-01 01:30:00+00:00', '2020-01-01 01:45:00+00:00',
               '2020-01-01 02:00:00+00:00', '2020-01-01 02:15:00+00:00',
               ...
               '2020-12-31 21:45:00+00:00', '2020-12-31 22:00:00+00:00',
               '2020-12-31 22:15:00+00:00', '2020-12-31 22:30:00+00:00',
               '2020-12-31 22:45:00+00:00', '2020-12-31 23:00:00+00:00',
               '2020-12-31 23:15:00+00:00', '2020-12-31 23:30:00+00:00',
               '2020-12-31 23:45:00+00:00', '2021-01-01 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=35137, freq='15T')

In [4]:
# df, pvlib, whatever Daten

In [5]:
# Timezones, resampling, missing values